# VacationPy
----
#### Author: Felipe Murillo
#### Date: May 2, 2020
#### Description: Use weather data from 500+ cities across the world to plan the perfect vacation
#### Inputs:  ./WeatherPy/output_data/cities.csv
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

# Import API key
sys.path.append('../')
from WeatherPy.config import gkey


### Input City Weather DataFrame


In [2]:
#Read in city weather data
city_weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv",index_col=0)
city_weather_df.head()

,Unnamed: 0.1,City,City Name,Country,Lng,Lat,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,alta floresta,Alta Floresta,BR,-56.09,-9.88,89.60,52.0,20.0,10.29
1,1,saskylakh,Saskylakh,RU,114.08,71.92,11.32,95.0,88.0,4.56
2,2,punta arenas,Punta Arenas,CL,-70.92,-53.15,48.20,76.0,40.0,9.17
3,3,wanning,Wanning,CN,110.40,18.80,74.64,87.0,19.0,6.82
4,4,zhigansk,Zhigansk,RU,123.37,66.77,20.55,97.0,100.0,7.99


In [3]:
# Store latitude and longitude in locations
city_locations = city_weather_df[["Lat","Lng"]]
city_locations.head()

,Lat,Lng
0,-9.88,-56.09
1,71.92,114.08
2,-53.15,-70.92
3,18.80,110.40
4,66.77,123.37


### Humidity Heatmap

In [4]:
#Configure gmaps
gmaps.configure(api_key=gkey)

In [5]:
# Plot humidity heatmap
fig = gmaps.figure(map_type="HYBRID",
                   center=(0, 0), 
                   zoom_level=1
                  )
heatmap_layer = gmaps.heatmap_layer(
    city_locations,
    weights = city_weather_df["Humidity (%)"],
    max_intensity = 150,
    point_radius = 10
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [6]:
# Create a filter for the following conditions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness
max_ideal_temp = 80 # deg F
min_ideal_temp = 70 # deg F
max_ideal_wind = 10 # mph
max_ideal_cloud = 0 # cloudiness (%)

perfect_weather = (city_weather_df["Max Temp (F)"] > min_ideal_temp) & \
                  (city_weather_df["Max Temp (F)"] < max_ideal_temp) & \
                  (city_weather_df["Wind Speed (mph)"] < max_ideal_wind) & \
                  (city_weather_df["Cloudiness (%)"] <= max_ideal_cloud)
ideal_cities = city_weather_df.loc[perfect_weather].reset_index()
ideal_cities

,index,Unnamed: 0.1,City,City Name,Country,Lng,Lat,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,22,23,sao filipe,São Filipe,CV,-24.50,14.90,77.32,62.0,0.0,7.92
1,180,193,linhares,Linhares,BR,-40.07,-19.39,72.57,80.0,0.0,3.87
2,189,202,pasighat,Pāsighāt,IN,95.33,28.07,70.77,67.0,0.0,3.44
3,194,210,tiznit,Tiznit Province,MA,-9.50,29.58,72.03,45.0,0.0,2.01
4,232,255,nsanje,Nsanje,MW,35.26,-16.92,70.41,88.0,0.0,4.14
5,241,265,abnub,Abnūb,EG,31.15,27.27,78.80,34.0,0.0,8.05
6,287,316,pisco,Pisco,PE,-76.22,-13.70,73.40,69.0,0.0,6.93
7,332,367,villa constitucion,Villa Constitución,AR,-60.33,-33.23,75.20,38.0,0.0,9.17
8,399,440,dubrajpur,Dubrājpur,IN,87.38,23.80,79.83,61.0,0.0,5.84
9,424,466,weinan,Weinan,CN,109.51,34.50,71.01,62.0,0.0,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_cities[["City Name","Country","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"]= ""
hotel_df["Hotel Lng"]= ""
hotel_df

/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,City Name,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
0,São Filipe,CV,14.90,-24.50,,,
1,Linhares,BR,-19.39,-40.07,,,
2,Pāsighāt,IN,28.07,95.33,,,
3,Tiznit Province,MA,29.58,-9.50,,,
4,Nsanje,MW,-16.92,35.26,,,
5,Abnūb,EG,27.27,31.15,,,
6,Pisco,PE,-13.70,-76.22,,,
7,Villa Constitución,AR,-33.23,-60.33,,,
8,Dubrājpur,IN,23.80,87.38,,,
9,Weinan,CN,34.50,109.51,,,


In [8]:
# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # assemble url and make API request
    print(f"Retrieving Results for Index: {index}, hotel in {row['City Name']}")
    
    # geocoordinates
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "Hotel"
    target_radius = 50000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': gkey
        }
    
    # Pull data from API
    hotel_data = requests.get(base_url, params=params).json()
    
    # extract results
    results = hotel_data['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
        hotel_df.loc[index, 'Hotel Lat'] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, 'Hotel Lng'] = results[0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index: 0, hotel in São Filipe


/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


------------
Retrieving Results for Index: 1, hotel in Linhares
------------
Retrieving Results for Index: 2, hotel in Pāsighāt
------------
Retrieving Results for Index: 3, hotel in Tiznit Province
------------
Retrieving Results for Index: 4, hotel in Nsanje
------------
Retrieving Results for Index: 5, hotel in Abnūb
------------
Retrieving Results for Index: 6, hotel in Pisco
------------
Retrieving Results for Index: 7, hotel in Villa Constitución
------------
Retrieving Results for Index: 8, hotel in Dubrājpur
------------
Retrieving Results for Index: 9, hotel in Weinan
------------
Retrieving Results for Index: 10, hotel in Jiexiu
------------
Retrieving Results for Index: 11, hotel in Maţāy
------------


In [9]:
# Show list of hotels in ideal cities and do a little clean up
hotel_df = hotel_df.rename(columns ={"Lat":"City Lat","Lng":"City Lng"})
hotel_df

,City Name,Country,City Lat,City Lng,Hotel Name,Hotel Lat,Hotel Lng
0,São Filipe,CV,14.90,-24.50,Hotel Miramar Fogo Brava,14.9043,-24.4999
1,Linhares,BR,-19.39,-40.07,Ibis,-19.8255,-40.2664
2,Pāsighāt,IN,28.07,95.33,OYO 65401 Hotel Poba,27.8239,95.2266
3,Tiznit Province,MA,29.58,-9.50,Hotel Des Touristes,29.6978,-9.73166
4,Nsanje,MW,-16.92,35.26,Shumba Lodge,-16.9295,35.2596
5,Abnūb,EG,27.27,31.15,بيت القديسة مريم St Marie House,27.1158,31.1707
6,Pisco,PE,-13.70,-76.22,"Hotel Paracas, a Luxury Collection Resort, Par...",-13.8363,-76.2542
7,Villa Constitución,AR,-33.23,-60.33,Nahuel Hotel,-32.9388,-60.6724
8,Dubrājpur,IN,23.80,87.38,Hotel Regal International,23.6871,86.9707
9,Weinan,CN,34.50,109.51,Hilton Xi'an,34.2646,108.966


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel locations
hotel_locations = [hotel_df.loc[hotel][['Hotel Lat','Hotel Lng']] for hotel in hotel_df.index]

# Create marker layer
marker_layer = gmaps.marker_layer(hotel_locations,info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(marker_layer)

#Display Map
fig

Figure(layout=FigureLayout(height='420px'))